In [ ]:
%matplotlib inline
import seaborn as sns
sns.set()

from datetime import datetime, date
import logging

from octo_spork.query import Range, In, Column, Query, And, EqualTo
from octo_spork.pandas_engine import PandasEngine
from octo_spork.resolution_tree import create_tree, run_tree, DataSources

logging.basicConfig(level=logging.INFO)

In [ ]:
from pedestrians import PedestrianSource


class FrontEndColumn(object):

    def __init__(self, column):
        self._column = column

    def isin(self, valueset):
        return In(self._column, valueset)

    def between(self, lower, upper):
        return Range(self._column, lower=lower, upper=upper, incl_lower=True, incl_upper=True)

    def __eq__(self, value):
        return EqualTo(self._column, value)


class PedestrianCounts(object):

    table = 'counts'

    datetime = FrontEndColumn(Column('counts', 'datetime'))
    sensor_id = FrontEndColumn(Column('counts', 'sensor_id'))
    hourly_count = FrontEndColumn(Column('counts', 'hourly_count'))

    columns = [datetime._column, sensor_id._column, hourly_count._column]

    sources = DataSources(
        source_order=['counts'],
        source_map={'counts': PedestrianSource()},
        source_tables={'counts': {'counts'}})

    def __init__(self, where=None):
        self._where = where

    def where(self, expression):
        if self._where is None:
            return self.__class__(expression)
        else:
            return self.__class__(And([self._where, expression]))

    def get(self):
        query = Query(table=self.table, select=self.columns, where=self._where)
        engine = PandasEngine()
        result_store = dict()
        tree = create_tree(query, self.sources)
        return run_tree(tree, self.sources, engine, result_store)

    def plot_timeseries(self, ts, value, group=None):
        if group is None:
            self.get().set_index(ts)[value].plot()
        else:
            self.get().set_index([ts, group]).unstack()[value].plot()

In [ ]:
# Create a dataset object filtered to our time range.
peds = PedestrianCounts()
peds = peds.where(peds.datetime.between(datetime(2016, 5, 1), datetime(2016, 5, 8)))

In [ ]:
# Filter to single sensor and plot results.
peds.where(peds.sensor_id == 1).plot_timeseries(ts='datetime', value='hourly_count')

In [ ]:
# Filter multi sensors and plot results.
peds.where(peds.sensor_id.isin([1, 2, 3])).plot_timeseries(ts='datetime', value='hourly_count', group='sensor_id')